In [18]:
from openai import OpenAI
import os
import json
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()

# Set parameters

name = "1"
model = "gpt-3.5-turbo"               # important options: "gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"
loops = 4
memory_limit = False
memory = 1                            # only in case of limited memory
stable_system_message = False
assistant_transform = False   
assistant_transform_type = "user"     # only in case of assistant transform, options: "user", "user-assistant", "system"
stream = False

first_prompt = [
  #{"role": "system", "content": ""},
  {"role": "user", "content": ""}
]

# Choose or create the right folder with json files

folder_name = f"{model}"
if memory_limit:
  folder_name += f" memory-{memory}"
if assistant_transform:
  folder_name += f" {assistant_transform_type}"
os.makedirs(folder_name, exist_ok=True)

for loop in range(loops):

  # Load completions from json, if continuing with a file generated in the past

  try:
    with open(f"{folder_name}/{name}.json", "r") as json_file:
      completions=json.load(json_file)

  # If not, create a new completions database

  except FileNotFoundError:
    completions = []

  # Collect messages generated so far

  messages = first_prompt
  for completion in completions:
      message = completion["choices"][0]["message"]
      messages.append({"role": message["role"], "content": message["content"]}   
      )

  # Pick prompt messages

  if (memory >= len(messages)) or not memory_limit:
      prompt_messages = messages
  else:
      #print(f"Memory smaller than the entire conversation. Forgetting {len(messages)-memory} messages")
      prompt_messages = []
      if stable_system_message:
        prompt_messages.append(first_prompt[0])
        print(prompt_messages)
        print("stable_system_message")
      prompt_messages = prompt_messages+messages[(len(messages)-memory):]

  # Transform assistant role to assistant and user, user, or system roles

  if assistant_transform:
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages):
        if prompt_messages[-i]["role"] == "assistant":
          prompt_messages[-i]["role"] = "user"
        i += 2
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type


  # Send to OpenAI API to get a new completion

  completion = client.chat.completions.create(
        model=model,
        messages = prompt_messages,
        stream = stream,
      )
  print(f"Prompt messages sent to OpenAI API:{prompt_messages}")
  if stream:
    all_completion_data = []
    content_string = ""
    for chunk in completion:
      all_completion_data.append(chunk)
      print(chunk.choices[0].delta.content or "", end="")
      if type(chunk.choices[0].delta.content) == str:
        content_string.join(chunk.choices[0].delta.content)
    completion = all_completion_data[0]
    delta = completion.choices[0].delta
    content = delta.content
    completion.choices[0] = delta
  else:
    print(completion.choices[0].message.content)

  # Update database and save it to the right json file

  completions.append(completion.model_dump())
  with open(f"{folder_name}/{name}.json", "w") as json_file:
      json.dump(completions,json_file)

 

Prompt messages sent to OpenAI API:[{'role': 'user', 'content': ''}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}]
Hello! How can I assist you today?
Prompt messages sent to OpenAI API:[{'role': 'user', 'content': ''}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': 'H

In [19]:
## Display data

# Load the JSON file

file_path = f"{folder_name}/{name}.json"

#file_path = f"gpt-3.5-turbo memory-1 user/2.json"

with open(file_path, 'r') as file:
    data = json.load(file)

# Normalize the JSON data in completions

normalized_data = pd.json_normalize(data, record_path='choices', meta=[
    'id', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 
    ['usage', 'completion_tokens'], ['usage', 'prompt_tokens'], ['usage', 'total_tokens']], sep='.',
    meta_prefix='', record_prefix='choices.')

# Make more important columns more visible

normalized_data = normalized_data[['choices.message.content', 'model', 
       'usage.completion_tokens', 'usage.prompt_tokens', 'usage.total_tokens', 'id', 
       'choices.finish_reason', 'choices.index', 'choices.logprobs', 
       'choices.message.role', 'choices.message.function_call', 'choices.message.tool_calls', 
       'created', 'object', 'service_tier', 'system_fingerprint'
       ]]

# Display

content_all = normalized_data["choices.message.content"]
last = content_all.iloc[-1]

print(file_path)
print(len(content_all))

for content in content_all:
   display_markdown(content, raw=True)

gpt-3.5-turbo/1.json
10


Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?

...

Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?

In [ ]:
completions[4]